In [52]:
# import datetime
# import time
import flywheel
import pandas as pd
import logging
import zipfile, os, shutil
import numpy as np

In [19]:
#Get Client
fw = flywheel.Client('flywheel.rc.colorado.edu:djEePPLCSOD8gbRQa7hoBvaPdGKDBtykGAMyorXcujMANwqQ77YPjyUIQ')
fw.get_config().site.api_url

# Instantiate a logger
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(levelname)s %(message)s')
log = logging.getLogger('root')

project_label = 'VMA'
# Get the project
project = fw.projects.find_one(f'label={project_label}')

In [20]:
project.stats.

{'age_groups': ['seniors'],
 'file_types': [None,
                'archive',
                'dicom',
                'tabular data',
                'text',
                'document',
                'nifti',
                'markup'],
 'measurements': ['B0', 'BOLD', 'T2*', 'T1', 'T2'],
 'modalities': ['MR'],
 'number_of': {'acquisition_files': 1245,
               'acquisitions': 408,
               'analyses': 98,
               'compliant_sessions': 0,
               'files': 2363,
               'jobs': 1031,
               'sessions': 24,
               'subjects': 24}}

In [34]:
analysis_list = []
for s in project.sessions.find():
    full_session = fw.get_session(s.id)
    for analys in full_session.analyses:
        if analys.gear_info.name == "bids-feat": #gear_info.name == 'bids-fmriprep' to download preprocessed files 
            analysis_list.append(analys.id)

['64e8f812a9f70014944a55ea',
 '64e917443482ad07372f1ca0',
 '64e917620397471e212f1cac',
 '64e917810397471e212f1cb0',
 '64e9179f27919747532f1d4c',
 '64e917bd27919747532f1d50',
 '64e917dbaea81c34f02f1cc5',
 '64e917fa0397471e212f1cb4',
 '64e9181827919747532f1d55',
 '64e9183627919747532f1d59',
 '64e91854aea81c34f02f1cc9',
 '64e9187327919747532f1d5e',
 '64e9189127919747532f1d62',
 '64e918afaea81c34f02f1ccd',
 '64e918cd27919747532f1d66',
 '64e918ec3482ad07372f1ca5',
 '64e9190a27919747532f1d6a',
 '64e919280397471e212f1cb8',
 '64e919463482ad07372f1ca9',
 '64e9196527919747532f1d6e',
 '64e919833482ad07372f1cae',
 '64e919a127919747532f1d72',
 '64ea41c93e46183de14a5658',
 '64e919c03482ad07372f1cb2']

In [36]:
path="/work/pi_rcowell_umass_edu/tsavalia/scratch/vma_fmri/feat/"
for cid in analysis_list:
    analysis = fw.get_container(cid)
    for fl in analysis.files:
        fl.download(path+fl['name'])

In [6]:

feat_files = os.listdir('feat/')
for sub in range(2, 25):
    print('sub started: ', sub)
    z = zipfile.ZipFile('feat/'+feat_files[sub-1])
    
    os.mkdir(f'feat/sub-{str(sub).zfill(2)}')
    z.extractall(f'feat/sub-{str(sub).zfill(2)}')    

sub started:  2
sub started:  3
sub started:  4
sub started:  5
sub started:  6
sub started:  7
sub started:  8
sub started:  9
sub started:  10
sub started:  11
sub started:  12
sub started:  13
sub started:  14
sub started:  15
sub started:  16
sub started:  17
sub started:  18
sub started:  19
sub started:  20
sub started:  21
sub started:  22
sub started:  23
sub started:  24


In [29]:
sub_dirs = [x for x in os.listdir('feat/') if x.startswith('sub')]

In [33]:
for sub_dir in sub_dirs[1:]:
    a = [x[0] for x in os.walk('feat/'+sub_dir)][4]
    shutil.move(a, f'feat/{sub_dir}/')


In [47]:
shutil.move('feat/sub-01/sub-01/ses-S1', 'feat/sub-01/')

'feat/sub-01/ses-S1'

In [46]:
os.listdir('feat/sub-01/sub-01/')

['ses-S1']

In [53]:
for sub_dir in sub_dirs[1:]:
    subject = np.array([x for x in os.listdir(f'feat/{sub_dir}') if x.startswith('sub')])[0]
    shutil.move(f'feat/{sub_dir}/{subject}/ses-S1', f'feat/{sub_dir}')